LSTM Нейросеть - Игорь

In [1]:
# Для нейросети
import sys
sys.path.append('..')
from config import Config
from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Flatten, Activation, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
%matplotlib inline

In [2]:
dataset_dir = "../datasets/dataset_numWords_20000__xLen_500_step_100_/"

xTrain = np.load(dataset_dir + "xTrain.npz")['arr_0']
yTrain = np.load(dataset_dir + "yTrain.npz")['arr_0']
xTest = np.load(dataset_dir + "xTest.npz")['arr_0']
yTest = np.load(dataset_dir + "yTest.npz")['arr_0']
xVal = np.load(dataset_dir + "xVal.npz")['arr_0']
yVal = np.load(dataset_dir + "yVal.npz")['arr_0']
# xTrain = np.load(Config.DATASET_X_TRAIN)
# yTrain = np.load(Config.DATASET_Y_TRAIN)
# xTest = np.load(Config.DATASET_X_TEST)
# yTest = np.load(Config.DATASET_Y_TEST)

# xVal, yVal = xTrain[:10000], yTrain[:10000]
# xTrain, yTrain = xTrain[10000:], yTrain[10000:]
# xBuffer, yBuffer = xTest, yTest
# xTest, yTest = xVal, yVal
# xVal, yVal = xBuffer, yBuffer
# # для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
print("Размерности тренировочного набора")
print(xTrain.shape)
print(yTrain.shape)
print()
print("Размерности тестового набора")
print(xTest.shape)
print(yTest.shape)
print()
print("Размерности val набора")
print(xVal.shape)
print(yVal.shape)

Размерности тренировочного набора
(4252, 500)
(4252, 2)

Размерности тестового набора
(425, 500)
(425, 2)

Размерности val набора
(84, 500)
(84, 2)


In [3]:
numWords = 20000
xLen = 500
Batch_Size = 100
step = 100

In [4]:
modelLSTM = Sequential()

modelLSTM.add(Embedding(numWords, step, input_length=xLen))
modelLSTM.add(SpatialDropout1D(0.2))
modelLSTM.add(BatchNormalization())

modelLSTM.add(LSTM(10))
modelLSTM.add(Dense(2, activation='softmax'))

modelLSTM.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

modelLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          2000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 500, 100)         0         
 ropout1D)                                                       
                                                                 
 batch_normalization (BatchN  (None, 500, 100)         400       
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 10)                4440      
                                                                 
 dense (Dense)               (None, 2)                 22        
                                                                 
Total params: 2,004,862
Trainable params: 2,004,662
Non-

In [5]:

historyLSTM = modelLSTM.fit(xTrain, 
                            yTrain, 
                            epochs=15,
                            batch_size=Batch_Size,
                            validation_data=(xVal, yVal),
                            use_multiprocessing=True)

Epoch 1/15


2021-12-16 16:18:09.134425: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


43/43 [==============================] - 7s 139ms/step - loss: 0.5778 - accuracy: 0.7307 - val_loss: 1.2418 - val_accuracy: 0.0000e+00
Epoch 2/15
43/43 [==============================] - 6s 133ms/step - loss: 0.4161 - accuracy: 0.8114 - val_loss: 1.3124 - val_accuracy: 0.0000e+00
Epoch 3/15
43/43 [==============================] - 6s 139ms/step - loss: 0.2948 - accuracy: 0.8756 - val_loss: 2.9362 - val_accuracy: 0.0000e+00
Epoch 4/15
43/43 [==============================] - 6s 144ms/step - loss: 0.1612 - accuracy: 0.9506 - val_loss: 4.5589 - val_accuracy: 0.0000e+00
Epoch 5/15
43/43 [==============================] - 6s 139ms/step - loss: 0.0731 - accuracy: 0.9847 - val_loss: 3.0881 - val_accuracy: 0.4286
Epoch 6/15
43/43 [==============================] - 6s 137ms/step - loss: 0.0362 - accuracy: 0.9962 - val_loss: 3.1673 - val_accuracy: 0.4524
Epoch 7/15
43/43 [==============================] - 6s 137ms/step - loss: 0.0185 - accuracy: 0.9988 - val_loss: 2.6214 - val_accuracy: 0.5714
E

In [6]:

# Проверка точности на тестовой выборке
rightAnswer = [0,0]
totalAnswer = [0,0]

# Получаем результаты распознавания для каждого класса
currPred = modelLSTM.predict(xTest)
# Определяем номер распознанного класса для каждого вектора
currOut = np.argmax(currPred, axis=1)
# Определяем правильный класс для каждого вектора
yOut = np.argmax(yTest, axis=1)

print(currPred.shape)
print(currOut.shape)
print(yOut.shape)
print()

for i in range(len(yOut)):
  predictA = currOut[i]
  rightA   = yOut[i]

  totalAnswer[rightA] += 1
  if predictA == rightA:
    rightAnswer[rightA] += 1

print("Точность распознавания текстов")
for i in range(2):
  print("{:12s}: {:3d} из {:3d} - {:3.2f}%".format(str(i), rightAnswer[i], totalAnswer[i], (rightAnswer[i]/totalAnswer[i]*100)))

(425, 2)
(425,)
(425,)

Точность распознавания текстов
0           : 311 из 346 - 89.88%
1           :  15 из  79 - 18.99%
